In [1]:
import pandas as pd
import numpy as np
import sys
import os

from typing import Dict, Any, Optional

if ".." not in sys.path:
    sys.path.append("..")
if "../contracts" not in sys.path:
    sys.path.append("../contracts")

from file_manager_helper import load_dataset_tag_contract, save_pandas_dataset
from reducers.dataset_contract_reducers import convert_dataset, modelling_filter
import matplotlib.pyplot as plt
import config

DATE_FORMAT: str = '%Y-%m-%d %H:%M:%S'
OUTPUT_DIR: str = "prepared_datasets"

def save_test_system_dataset(
    dataset: pd.DataFrame,
    dataset_tag: str,
    output_dataset_name: str):

    # Save data for testing the system
    real_time_sample = dataset.tail(20000)
    real_time_sample.index.rename("timestamp", inplace=True)
    real_time_sample.to_csv(os.path.join(OUTPUT_DIR, dataset_tag, f"BAT_realtime_dataset_{output_dataset_name}.csv"))

def convert_and_save_dataset(
    dataset: pd.DataFrame,
    dataset_tag: str,
    output_dataset_name: str,
    output_dataset_name_suffix: str,
    model_converters: Optional[Dict[str, Any]] = None,
    model_converters_association_tags: Optional[Dict[str, Any]] = None,
    act_on_na: bool = True):

    # Loading the dataset contract
    contract = load_dataset_tag_contract(
        contract_alias=dataset_tag,
         directory=os.path.join("..", config.ZIP_DATASET_CONTRACTS_DIRECTORY, dataset_tag))

    # Convert the dataset
    converted_dataset, dataset = convert_dataset(
        contract=contract,
        dataset=dataset,
        model_converters=model_converters,
        model_converters_association_tags=model_converters_association_tags,
        act_on_na=act_on_na)
    
    # Filter based on conditions of boiler being active
    #converted_dataset = modelling_filter(
    #                        contract=contract,
    #                        dataset=converted_dataset)

    # Plotting the efficiency
    if "Efficiency" in set(converted_dataset.columns):
        efficiency_values = converted_dataset["Efficiency"].values

        plt.plot(range(len(efficiency_values)), efficiency_values)
        plt.show()
        
    converted_dataset.index.names=["date_rec"]
    # Prepare the converted dataset for saving
    dataset = pd.melt(converted_dataset.reset_index(), id_vars='date_rec', value_vars=converted_dataset.columns, var_name='address_no', value_name='value')
    #dataset.rename(columns={'index': 'date_rec'}, inplace=True)
    # Make sure that the date is in the correct format by converting it.
    dataset['date_rec'] = dataset['date_rec'].dt.strftime(DATE_FORMAT)
    # Remove empty values from the dataset
    dataset['value'].replace('', np.nan, inplace=True)
    dataset.dropna(subset=['value'], inplace=True)
    
    
    # Save data for training the models
    save_pandas_dataset(dataset=dataset, dataset_tag=output_dataset_name, dataset_suffix=output_dataset_name_suffix, directory=OUTPUT_DIR)
    
def concatenate_daily(dir_path: str):
    dataset_to_concat = []
    
    for dir in os.listdir(dir_path):
        daily_df = pd.read_csv(os.path.join(dir_path,dir))
        # Appending dataframes to concatenate
        dataset_to_concat.append(daily_df)
    total_df = pd.concat(dataset_to_concat)
    return total_df

2022-09-24 10:16:51.282023: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-24 10:16:51.282052: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# PCT 1

In [ ]:
#with open('original_format/efficiency_modelling/PCT1/pct_1_efficiency_model.pickle', 'rb') as handle:
#    pct_1_efficiency_calculator = pickle.load(handle)

In [ ]:
dataset_1 = pd.read_csv("provided_data/datasets/PCT1.csv", parse_dates= ["timestamp"])
dataset_1.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_1.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("PCT1_for_sanity_checking_1.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT1_DATA_TAG,
    output_dataset_name="pct1")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT1_DATA_TAG,
    output_dataset_name=config.PCT1_DATA_TAG,
    output_dataset_name_suffix="1",
    act_on_na=True)

In [2]:
dataset_1 = pd.read_csv("provided_data/datasets/PCT1_updt.csv", parse_dates= ["timestamp"])
dataset_1.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_1.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("PCT1_for_sanity_checking_1.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT1_DATA_TAG,
    output_dataset_name="pct1")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT1_DATA_TAG,
    output_dataset_name=config.PCT1_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)

# PCT2 

In [ ]:
dataset_4 = pd.read_csv("provided_data/datasets/PCT2.csv", parse_dates= ["timestamp"])
dataset_4.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_4.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("PCT2_for_sanity_checking_1.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT2_DATA_TAG,
    output_dataset_name="pct2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT2_DATA_TAG,
    output_dataset_name=config.PCT2_DATA_TAG,
    output_dataset_name_suffix="1",
    act_on_na=True)

In [2]:
dataset_4 = pd.read_csv("provided_data/datasets/PCT2_updt.csv", parse_dates= ["timestamp"])
dataset_4.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_4.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("PCT2_for_sanity_checking_1.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT2_DATA_TAG,
    output_dataset_name="pct2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.PCT2_DATA_TAG,
    output_dataset_name=config.PCT2_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)

# RBG 1

In [ ]:
#with open('original_format/efficiency_modelling/RBG1/rbg_1_efficiency_model.pickle', 'rb') as handle:
#    rbg_1_efficiency_calculator = pickle.load(handle)

In [ ]:
dataset_2 = pd.read_csv("provided_data/datasets/RBG1/UJRBG2 2022-06-20.csv", parse_dates= ["timestamp"])
dataset_2.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_2.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("RBG1_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.RBG1_DATA_TAG,
    output_dataset_name="rbg1")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.RBG1_DATA_TAG,
    output_dataset_name=config.RBG1_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)

# RBG 2

In [ ]:
#with open('original_format/efficiency_modelling/RBG2/rbg_1_efficiency_model.pickle', 'rb') as handle:
#    rbg_3_efficiency_calculator = pickle.load(handle)

In [ ]:
dataset_3 = pd.read_csv("provided_data/datasets/RBG2.csv", parse_dates= ["timestamp"])
dataset_3.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_3.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("RBG2_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.RBG2_DATA_TAG,
    output_dataset_name="rbg2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.RBG2_DATA_TAG,
    output_dataset_name=config.RBG2_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=False)

# KTT 1

## KTT 1.1

In [3]:
dataset_5 = concatenate_daily("provided_data/datasets/KTT1/UJKTT_1/model_data")
dataset_5["f_timestamp"] = pd.to_datetime(dataset_5["f_timestamp"]) 
dataset_5 = dataset_5.sort_values(by="f_timestamp")
dataset_5.rename(columns={"f_timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_5.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("KTT1_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT1_DATA_TAG,
    output_dataset_name="ktt1")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT1_DATA_TAG,
    output_dataset_name=config.KTT1_DATA_TAG,
    output_dataset_name_suffix="1",
    act_on_na=False)

KeyboardInterrupt: 

## KTT 1.2 (compensating for KALTIM1.SIGNAL.AI.10HBK31CQ101 unavailability)

In [ ]:
dataset_7 = concatenate_daily("provided_data/datasets/KTT1/KTT1_additional_data_1/KTT1_13022022_12042022")
dataset_7["f_timestamp"] = pd.to_datetime(dataset_7["f_timestamp"]) 
dataset_7 = dataset_7.sort_values(by="f_timestamp")
dataset_7.rename(columns={"f_timestamp":"date_rec"}, inplace=True)
sensor_df = dataset_7.set_index("date_rec")

# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("KTT1_for_sanity_checking_2.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT1_DATA_TAG,
    output_dataset_name="ktt1")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT1_DATA_TAG,
    output_dataset_name=config.KTT1_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)

## KTT 1.3 (combination of the two)

In [ ]:
KTT_11 = pd.read_csv("prepared_datasets/dataset_KTT1_1.csv")
KTT_12 = pd.read_csv("prepared_datasets/dataset_KTT1_2.csv")
dataset_8 = pd.concat([KTT_11, KTT_12], ignore_index=True)
dataset_8.to_csv("prepared_datasets/dataset_KTT1_3.csv")

# KTT 2

# KTT 2.1

In [2]:
dataset_6 = concatenate_daily("provided_data/datasets/KTT2/KTT2_additional_data_1/KTT2_13022022_12042022")
dataset_6["f_timestamp"] = pd.to_datetime(dataset_6["f_timestamp"]) 
dataset_6 = dataset_6.sort_values(by="f_timestamp")
dataset_6.rename(columns={"f_timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_6.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("KTT2_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name="ktt2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name=config.KTT2_DATA_TAG,
    output_dataset_name_suffix="1",
    act_on_na=True)

# KTT 2.2

In [4]:
dataset_11 = concatenate_daily("provided_data/datasets/KTT2/UJKTT_2/model_data")
dataset_11["f_timestamp"] = pd.to_datetime(dataset_11["f_timestamp"]) 
dataset_11 = dataset_11.sort_values(by="f_timestamp")
dataset_11.rename(columns={"f_timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_11.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("KTT2_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name="ktt2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name=config.KTT2_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)

# KTT 2.3

In [6]:
KTT_21 = pd.read_csv("prepared_datasets/dataset_KTT2_1.csv")
KTT_22 = pd.read_csv("prepared_datasets/dataset_KTT2_2.csv")
dataset_12 = pd.concat([KTT_21, KTT_22], ignore_index=True)
dataset_12.to_csv("prepared_datasets/dataset_KTT2_3.csv")

# KTT 2.4

In [2]:
dataset_13 = concatenate_daily("provided_data/datasets/KTT2/KTT2_20220904_20220923")
dataset_13["f_timestamp"] = pd.to_datetime(dataset_13["f_timestamp"]) 
dataset_13 = dataset_13.sort_values(by="f_timestamp")
dataset_13.rename(columns={"f_timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_13.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("KTT2_for_sanity_checking_4.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name="ktt2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.KTT2_DATA_TAG,
    output_dataset_name=config.KTT2_DATA_TAG,
    output_dataset_name_suffix="4",
    act_on_na=True)

# KTT 2.5

In [3]:
KTT_23 = pd.read_csv("prepared_datasets/dataset_KTT2_3.csv")
KTT_24 = pd.read_csv("prepared_datasets/dataset_KTT2_4.csv")
dataset_14 = pd.concat([KTT_23, KTT_24], ignore_index=True)
dataset_14.to_csv("prepared_datasets/dataset_KTT2_5.csv")

# AMG 1

In [ ]:
dataset_9 = pd.read_csv("provided_data/datasets/AMG1/AMG1_new.csv", parse_dates=["timestamp"])
pa_pressure_dataset_9 = pd.read_csv("provided_data/datasets/AMG1/ALA30CP101.csv", parse_dates=["timestamp"])
pa_pressure_dataset_9 = pa_pressure_dataset_9.sort_values(by="timestamp")
dataset_9 = dataset_9.sort_values(by="timestamp")
dataset_9["ALA30CP101"] = pa_pressure_dataset_9["ALA30CP101"]
dataset_9.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_9.set_index("date_rec")
#Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("AMG1_for_sanity_checking.csv")

save_test_system_dataset(
   dataset=sensor_df,
   dataset_tag=config.AMG1_DATA_TAG,
   output_dataset_name="amg1")

convert_and_save_dataset(
   dataset=sensor_df,
   dataset_tag=config.AMG1_DATA_TAG,
   output_dataset_name=config.AMG1_DATA_TAG,
   output_dataset_name_suffix="2",
   act_on_na=True)

In [ ]:
sensor_df["ALA84CF1_1"].isna().sum()

# AMG 2

In [2]:
dataset_10 = pd.read_csv("provided_data/datasets/AMG2/AMG2_new.csv", parse_dates=["timestamp"])
pa_pressure_dataset_10 = pd.read_csv("provided_data/datasets/AMG2/BLA30CP101.csv", parse_dates=["timestamp"]) 
pa_pressure_dataset_10 = pa_pressure_dataset_10.sort_values(by="timestamp")
dataset_10 = dataset_10.sort_values(by="timestamp")
dataset_10["BLA30CP101"] = pa_pressure_dataset_10["BLA30CP101"]
dataset_10.rename(columns={"timestamp":"date_rec"}, inplace=True)

sensor_df = dataset_10.set_index("date_rec")
# Drop NAs only if for a row there are only NANs
sensor_df = sensor_df.dropna(how="all")
sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]

sensor_df.to_csv("AMG2_for_sanity_checking.csv")

save_test_system_dataset(
    dataset=sensor_df,
    dataset_tag=config.AMG2_DATA_TAG,
    output_dataset_name="amg2")

convert_and_save_dataset(
    dataset=sensor_df,
    dataset_tag=config.AMG2_DATA_TAG,
    output_dataset_name=config.AMG2_DATA_TAG,
    output_dataset_name_suffix="2",
    act_on_na=True)